In [1239]:
"""
скрипт для обработки данных мониторинга трудоустройства
"""


'\nскрипт для обработки данных мониторинга трудоустройства\n'

In [1240]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
import copy
import math
import os

warnings.simplefilter(action='ignore', category=UserWarning)

In [1241]:
def check_data(cell):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0


In [1242]:
def check_data_note(cell):
    if cell is np.nan:
        return 'Не заполнено'
    return str(cell)


In [1243]:
def check_first_error(df:pd.DataFrame,name_file):
    """
    Функция для проверки гр. 09 и гр. 10 < гр. 08 
    """
    
    df['Сумма'] = df['09'] + df['10']
    # Проводим проверку
    df['Результат'] = df['08'] >= df['Сумма']
    # заменяем булевые значения на понятные
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно' )
     # получаем датафрейм с ошибками и извлекаем индекс
    df=df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist() # делаем список
    finish_lst_index = list(map(lambda x:x +9,raw_lst_index))
    finish_lst_index = list(map(lambda x:f'Строка {str(x)}',finish_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = 'гр. 09 и гр. 10 < гр. 08 '
    return temp_error_df


def check_second_error(df:pd.DataFrame,name_file):
    """    
    Функция для проверки правильности введеденных данных
    (гр. 07= гр.08 + сумма(с гр.11 по гр.32))
    :param df: копия датафрейма с данными из файла поо
    :return:датафрейм с ошибками
    """
    # конвертируем в инт
    all_sum_cols = list(df)
    # удаляем колонки 07, 09, 10
    all_sum_cols.remove('07')
    all_sum_cols.remove('09')
    all_sum_cols.remove('10')
    # получаем сумму колонок 08, 11:32
    df['Сумма'] = df[all_sum_cols].sum(axis=1)
    # Проводим проверку
    df['Результат'] = df['07'] == df['Сумма']
    # заменяем булевые значения на понятные
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    # получаем датафрейм с ошибками и извлекаем индекс
    df=df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist() # делаем список
    finish_lst_index = list(map(lambda x:x +9,raw_lst_index))
    finish_lst_index = list(map(lambda x:f'Строка {str(x)}',finish_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = 'Не совпадает сумма гр. 07 гр.08 + сумма(с гр.11 по гр.32)'
    return temp_error_df


def check_third_error(df:pd.DataFrame,name_file,border):
    """    
    Функция для проверки правильности введеденных данных
    стр. 06 = стр. 02 + стр. 04
    :param foo_df: копия датафрейма с данными из файла поо
    :return:датафрейм с ошибками
    """
    foo_df = pd.DataFrame(columns=['02','04','06',])

    # Добавляем данные в датафрейм
    foo_df['02'] = df.iloc[1,:]
    foo_df['04'] = df.iloc[3,:]
    foo_df['06'] = df.iloc[5,:]
    foo_df['Сумма'] = foo_df['02'] + foo_df['04']
    foo_df['Результат'] = foo_df['06'] == foo_df['Сумма']
    foo_df['Результат'] = foo_df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')

    foo_df=foo_df[foo_df['Результат'] == 'Неправильно'].reset_index()
    temp_error_df = pd.DataFrame(columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = foo_df['index'].tolist() # делаем список
    finish_lst_index = list(map(lambda x:f'Диапазон строк {border+9} - {border+23} колонка {str(x)}',raw_lst_index))

    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = ' Не совпадает сумма стр. 06 = стр. 02 + стр. 04 '
    return temp_error_df

    

def check_fourth_error(df:pd.DataFrame,name_file,border):
    """    
    Функция для проверки правильности введеденных данных
    стр. 06 = стр.07 + стр.08 + стр.09 + стр.10 + стр.11 + стр.12 + стр. 13

    :param foo_df: копия датафрейма с данными из файла поо
    :return:датафрейм с ошибками
    """
    

    foo_df = pd.DataFrame(columns=['06','07','08','09','10','11','12','13'])

    # Добавляем данные в датафрейм
    foo_df['06'] = df.iloc[5,:]
    foo_df['07'] = df.iloc[6,:]
    foo_df['08'] = df.iloc[7,:]
    foo_df['09'] = df.iloc[8,:]
    foo_df['10'] = df.iloc[9,:]
    foo_df['11'] = df.iloc[10,:]
    foo_df['12'] = df.iloc[11,:]
    foo_df['13'] = df.iloc[12,:]


    sum_col = ['07','08','09','10','11','12','13']
    foo_df['Сумма'] = foo_df[sum_col].sum(axis=1)
    foo_df['Результат'] = foo_df['06'] == foo_df['Сумма']
    foo_df['Результат'] = foo_df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    foo_df=foo_df[foo_df['Результат'] == 'Неправильно'].reset_index()


    temp_error_df = pd.DataFrame(columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = foo_df['index'].tolist() # делаем список
    finish_lst_index = list(map(lambda x:f'Диапазон строк {border+9} - {border+23} колонка {str(x)}',raw_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = ' Не совпадает сумма стр. 06 = стр.07 + стр.08 + стр.09 + стр.10 + стр.11 + стр.12 + стр. 13 '
    
    return temp_error_df
    

def check_fifth_error(df:pd.DataFrame,name_file,border):
    """    
    Функция для проверки правильности введеденных данных
    стр. 14<=стр. 06, стр. 14<=стр 05 (<= означает "меньше или равно")
    :param foo_df: копия датафрейма с данными из файла поо
    :return:датафрейм с ошибками
    """
    foo_df = pd.DataFrame(columns=['05','06','14'])

    # Добавляем данные в датафрейм
    foo_df['05'] = df.iloc[4,:]
    foo_df['06'] = df.iloc[5,:]
    foo_df['14'] = df.iloc[13,:]




    foo_df['Результат'] = (foo_df['14'] <= foo_df['05']) & (foo_df['14'] <= foo_df['06'])
    foo_df['Результат'] = foo_df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    foo_df=foo_df[foo_df['Результат'] == 'Неправильно'].reset_index()


    temp_error_df = pd.DataFrame(columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = foo_df['index'].tolist() # делаем список, прибавляем для того чтобы номера строк совпадали с строками в файле
    finish_lst_index = list(map(lambda x:f'В диапазоне строк {border+9} - {border+23} в колонке {str(x)}',raw_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = ' Не совпадает сумма стр. 14<=стр. 06, стр. 14<=стр 05'
    return temp_error_df
    

In [1244]:
def check_error(df:pd.DataFrame):
    """
    Функция для проверки данных
    """
    # создаем датафрейм для регистрации ошибок
    error_df = pd.DataFrame(columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки',])
    df = df.iloc[:,6:32]
    df = df.applymap(check_data)

    # получаем количество датафреймов
    quantity = df.shape[0] // 15
    # счетчик для обработанных строк
    border = 0
    for i in range(1,quantity+1):
        temp_df = df.iloc[border:border +15,:]
        # Проводим проверку гр. 09 и гр. 10 <= гр. 08
        first_error_df = check_first_error(temp_df.copy(),name_file)
         # добавляем результат проверки в датафрейм
        error_df = pd.concat([error_df,first_error_df],axis=0,ignore_index=True)


        # Проводим проверку гр. 07= гр.08 + сумма(с гр.11 по гр.32)
        second_error_df = check_second_error(temp_df.copy(),name_file)
        # добавляем результат проверки в датафрейм
        error_df = pd.concat([error_df,second_error_df],axis=0,ignore_index=True)

        # Проводим проверку стр. 06 = стр. 02 + стр. 04
        third_error_df = check_third_error(temp_df.copy(),name_file,border)
        # добавляем результат проверки в датафрейм
        error_df = pd.concat([error_df,third_error_df],axis=0,ignore_index=True)


        # Проводим проверку стр. 06 = стр.07 + стр.08 + стр.09 + стр.10 + стр.11 + стр.12 + стр. 13
        fourth_error_df = check_fourth_error(temp_df.copy(),name_file,border)
        # добавляем результат проверки в датафрейм
        error_df = pd.concat([error_df,fourth_error_df],axis=0,ignore_index=True)

        # Проводим проверку стр. 14<=стр. 06, стр. 14<=стр 05 (<= означает "меньше или равно")
        fifth_error_df = check_fifth_error(temp_df.copy(),name_file,border)
        # добавляем результат проверки в датафрейм
        error_df = pd.concat([error_df,fifth_error_df],axis=0,ignore_index=True)
        # прибавляем border
        border += 15
    # Возвращаем датафрейм с ошибками
    return error_df


    
    

In [1245]:
path_to_files ='data/poo'
path_to_end_folder = 'data/temp'


In [1246]:
# создаем словарь верхнего уровня для каждого поо
high_level_dct = {}
# создаем датафрейм для регистрации ошибок
error_df = pd.DataFrame(columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки',])

In [1247]:
for file in os.listdir(path_to_files):
    name_file = file.split('.xlsx')[0]
    print(name_file)
    df = pd.read_excel(f'{path_to_files}/{file}',skiprows=7,dtype=str)
    if df.shape[0] % 15 != 0:
        
        temp_error_df = pd.DataFrame(data=[[f'{name_file}','','КОЛИЧЕСТВО СТРОК С ДАННЫМИ НЕ КРАТНО 15 !!!']],
                                    columns=['Название файла','Строка или колонка с ошибкой','Описание ошибки'])
        error_df = pd.concat([error_df,temp_error_df],axis=0,ignore_index=True)
        continue
    df.columns = list(map(str,df.columns))
    # Заполняем пока пропуски в 15 ячейке для каждой специальности
    df['06'] = df['06'].fillna('15 ячейка')
    
    # Проводим проверку на корректность данных, отправляем копию датафрейма
    file_error_df = check_error(df.copy())
    error_df = pd.concat([error_df,file_error_df],axis=0,ignore_index=True)
    
    # очищаем от нан и возможнных пустых пробелов
    code_spec =[spec for spec in df['03'].unique() if spec is not np.nan]
    code_spec =[spec for spec in code_spec if spec != ' ']
    
    
    # Создаем список для строк
    row_cat = [f'Строка {i}' for i in range(1,16)]
    # Создаем список для колонок
    column_cat = [f'Колонка {i}' for i in range(7,34)]

    # Создаем словарь нижнего уровня содержащий в себе все данные для каждой специальности
    spec_dict = {}
    for row in row_cat:
        spec_dict[row] = {key:0 for key in column_cat}
    
    
    # Изменяем последний ключ на строковый поскольку там будут хранится примечания
    for row,value in spec_dict.items():
        for col,data in value.items():
            if col == 'Колонка 33':
                spec_dict[row][col] = ''
    # Создаем словарь среднего уровня содержащй данные по всем специальностям
    poo_dct = {key:copy.deepcopy(spec_dict) for key in code_spec}

    high_level_dct[name_file] = copy.deepcopy(poo_dct)

    """
    В итоге получается такая структура
    {БРИТ:{13.01.10:{Строка 1:{Колонка 1:0}}},ТСИГХ:{22.01.10:{Строка 1:{Колонка 1:0}}}}

    """

    current_code = 'Ошибка проверьте правильность заполнения кодов специальностей' # чекбокс для проверки заполнения кода специальности

    idx_row = 1 # счетчик обработанных строк

    # Итерируемся по полученному датафрейму через itertuples
    for row in df.itertuples():
        # если счетчик колонок больше 15 то уменьшаем его до единицы
        if idx_row >15:
            idx_row = 1

        #print(row)
        # Проверяем на незаполненные ячейки и ячейки заполненные пробелами
        if (row[3] is not np.nan) and (row[3] != ' '):
            # если значение ячейки отличается от текущего кода специальности то обновляем значение текущего кода
            if row[3] != current_code:
                current_code = row[3]
            
        data_row = row[7:34] # получаем срез с нужными данными
        
        for idx_col,value in enumerate(data_row,start=1):
            if idx_col + 6 == 33:
                # сохраняем примечания в строке
                high_level_dct[name_file][current_code][f'Строка {idx_row}'][f'Колонка {idx_col + 6}'] = f'{name_file} {check_data_note(value)};'

            else:                
                high_level_dct[name_file][current_code][f'Строка {idx_row}'][f'Колонка {idx_col + 6}'] += check_data(value)
        
        
        idx_row += 1
                
            


    
    
    
    


Брит - 2022
Ошибка 3
Резерв
ТСИГХ мониторониг


In [1248]:
error_df.to_excel('errors.xlsx',index=False)

,Название файла,Строка или колонка с ошибкой,Описание ошибки
0,Брит - 2022,Строка 9,гр. 09 и гр. 10 < гр. 08
1,Брит - 2022,Строка 10,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
2,Брит - 2022,Строка 11,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
3,Брит - 2022,Строка 12,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
4,Брит - 2022,Строка 13,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
5,Брит - 2022,Строка 14,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
6,Брит - 2022,Строка 15,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
7,Брит - 2022,Строка 16,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
8,Брит - 2022,Строка 17,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...
9,Брит - 2022,Строка 18,Не совпадает сумма гр. 07 гр.08 + сумма(с гр.1...


In [1249]:
high_level_dct['Брит - 2022']['43.01.06']['Строка 1']

{'Колонка 7': 21,
 'Колонка 8': 8,
 'Колонка 9': 7,
 'Колонка 10': 100,
 'Колонка 11': 0,
 'Колонка 12': 0,
 'Колонка 13': 9,
 'Колонка 14': 3,
 'Колонка 15': 0,
 'Колонка 16': 1,
 'Колонка 17': 0,
 'Колонка 18': 0,
 'Колонка 19': 0,
 'Колонка 20': 0,
 'Колонка 21': 0,
 'Колонка 22': 0,
 'Колонка 23': 0,
 'Колонка 24': 0,
 'Колонка 25': 0,
 'Колонка 26': 0,
 'Колонка 27': 0,
 'Колонка 28': 0,
 'Колонка 29': 0,
 'Колонка 30': 0,
 'Колонка 31': 0,
 'Колонка 32': 0,
 'Колонка 33': 'Брит - 2022 пример примечания;'}

In [1250]:
high_level_dct['ТСИГХ мониторониг']['43.01.06']['Строка 1']

{'Колонка 7': 21,
 'Колонка 8': 8,
 'Колонка 9': 7,
 'Колонка 10': 55,
 'Колонка 11': 0,
 'Колонка 12': 0,
 'Колонка 13': 0,
 'Колонка 14': 0,
 'Колонка 15': 0,
 'Колонка 16': 1,
 'Колонка 17': 0,
 'Колонка 18': 0,
 'Колонка 19': 0,
 'Колонка 20': 0,
 'Колонка 21': 0,
 'Колонка 22': 0,
 'Колонка 23': 0,
 'Колонка 24': 0,
 'Колонка 25': 0,
 'Колонка 26': 0,
 'Колонка 27': 0,
 'Колонка 28': 0,
 'Колонка 29': 0,
 'Колонка 30': 0,
 'Колонка 31': 0,
 'Колонка 32': 0,
 'Колонка 33': 'ТСИГХ мониторониг пример примечания;'}

In [1251]:
high_level_dct['ошибка 2']['43.01.06']['Строка 1']

KeyError: 'ошибка 2'

In [ ]:

# получаем уникальные специальности
all_spec_code = set()
for poo,spec in high_level_dct.items():
    for code_spec,data in spec.items():
        all_spec_code.add(code_spec)



In [ ]:
itog_df = {key:copy.deepcopy(spec_dict) for key in all_spec_code}

In [ ]:
# Складываем результаты
for poo,spec in high_level_dct.items():
    for code_spec,data in spec.items():
        for row,col_data in data.items():
            for col,value in col_data.items():
                if col == 'Колонка 33':
                    itog_df[code_spec][row][col] += check_data_note(value)
                else:
                    itog_df[code_spec][row][col] += value

In [ ]:
itog_df['43.01.06']['Строка 1']['Колонка 33']

In [ ]:
out_df =pd.DataFrame.from_dict(itog_df, orient='index')

In [ ]:
stack_df = out_df.stack()
    # название такое выбрал потому что было лень заменять значения из блокнота юпитера
frame = stack_df.to_frame()

In [ ]:
frame

In [ ]:
frame['Всего'] = frame[0].apply(lambda x:x.get('Колонка 7'))
frame['Трудоустроены (по трудовому договору, договору ГПХ в соответствии с трудовым законодательством, законодательством  об обязательном пенсионном страховании)'] = frame[0].apply(lambda x:x.get('Колонка 8'))
frame['В том числе (из трудоустроенных): в соответствии с освоенной профессией, специальностью (исходя из осуществляемой трудовой функции)'] = frame[0].apply(lambda x:x.get('Колонка 9'))
frame['В том числе (из трудоустроенных): работают на протяжении не менее 4-х месяцев на последнем месте работы'] = frame[0].apply(lambda x:x.get('Колонка 10'))
frame['Индивидуальные предприниматели'] = frame[0].apply(lambda x:x.get('Колонка 11'))
frame['Самозанятые (перешедшие на специальный налоговый режим  - налог на профессио-нальный доход)'] = frame[0].apply(lambda x:x.get('Колонка 12'))
frame['Продолжили обучение'] = frame[0].apply(lambda x:x.get('Колонка 13'))
frame['Проходят службу в армии по призыву'] = frame[0].apply(lambda x:x.get('Колонка 14'))
frame['Проходят службу в армии на контрактной основе, в органах внутренних дел, Государственной противопожарной службе, органах по контролю за оборотом наркотических средств и психотропных веществ, учреждениях и органах уголовно-исполнительной системы, войсках национальной гвардии Российской Федерации, органах принудительного исполнения Российской Федерации*'] = frame[0].apply(lambda x:x.get('Колонка 15'))
frame['Находятся в отпуске по уходу за ребенком'] = frame[0].apply(lambda x:x.get('Колонка 16'))
frame['Неформальная занятость (нелегальная)'] = frame[0].apply(lambda x:x.get('Колонка 17'))
frame['Зарегистрированы в центрах занятости в качестве безработных (получают пособие по безработице) и не планируют трудоустраиваться'] = frame[0].apply(lambda x:x.get('Колонка 18'))
frame['Не имеют мотивации к трудоустройству (кроме зарегистрированных в качестве безработных) и не планируют трудоустраиваться, в том числе по причинам получения иных социальных льгот '] = frame[0].apply(lambda x:x.get('Колонка 19'))
frame['Иные причины нахождения под риском нетрудоустройства'] = frame[0].apply(lambda x:x.get('Колонка 20'))
frame['Смерть, тяжелое состояние здоровья'] = frame[0].apply(lambda x:x.get('Колонка 21'))
frame['Находятся под следствием, отбывают наказание'] = frame[0].apply(lambda x:x.get('Колонка 22'))
frame['Переезд за пределы Российской Федерации (кроме переезда в иные регионы - по ним регион должен располагать сведениями)'] = frame[0].apply(lambda x:x.get('Колонка 23'))
frame['Не могут трудоустраиваться в связи с уходом за больными родственниками, в связи с иными семейными обстоятельствами'] = frame[0].apply(lambda x:x.get('Колонка 24'))
frame['Выпускники из числа иностранных граждан, которые не имеют СНИЛС'] = frame[0].apply(lambda x:x.get('Колонка 25'))
frame['Иное (в первую очередь выпускники распределяются по всем остальным графам. Данная графа предназначена для очень редких случаев. Если в нее включено более 1 из 200 выпускников - укажите причины в гр. 33 '] = frame[0].apply(lambda x:x.get('Колонка 26'))
frame['будут трудоустроены'] = frame[0].apply(lambda x:x.get('Колонка 27'))
frame['будут осуществлять предпринимательскую деятельность'] = frame[0].apply(lambda x:x.get('Колонка 28'))
frame['будут самозанятыми'] = frame[0].apply(lambda x:x.get('Колонка 29'))
frame['будут призваны в армию'] = frame[0].apply(lambda x:x.get('Колонка 30'))
frame['будут в армии на контрактной основе, в органах внутренних дел, Государственной противопожарной службе, органах по контролю за оборотом наркотических средств и психотропных веществ, учреждениях и органах уголовно-исполнительной системы, войсках национальной гвардии Российской Федерации, органах принудительного исполнения Российской Федерации*'] = frame[0].apply(lambda x:x.get('Колонка 31'))
frame['будут продолжать обучение'] = frame[0].apply(lambda x:x.get('Колонка 32'))
frame['Принимаемые меры по содействию занятости (тезисно - вид меры, охват выпускников мерой)'] = frame[0].apply(lambda x:x.get('Колонка 33'))

In [ ]:
finish_df = frame.drop([0],axis=1)

In [ ]:
finish_df=finish_df.reset_index()

In [ ]:
finish_df.rename(columns={'level_0':'Код специальности','level_1':'Наименование показателей (категория выпускников)'},inplace=True)

In [ ]:
dct = {'Строка 1':'Всего (общая численность выпускников)','Строка 2':'из общей численности выпускников (из строки 01): лица с ОВЗ','Строка 3':'из числа лиц с ОВЗ (из строки 02): инвалиды и дети-инвалиды',
           'Строка 4':'Инвалиды и дети-инвалиды (кроме учтенных в строке 03)','Строка 5':'Имеют договор о целевом обучении','Строка 6':'Автосумма строк 02 и 04 - Всего (общая численность выпускников из числа лиц с ОВЗ, инвалидов и детей-инвалидов) '
           ,'Строка 7':'из общей численности выпускников из числа лиц с ОВЗ, инвалидов и детей-инвалидов (из строки 06): с нарушениями: зрения','Строка 8':'слуха','Строка 9':'опорно-двигательного аппарата',
           'Строка 10':'тяжелыми нарушениями речи','Строка 11':'задержкой психического развития','Строка 12':'расстройствами аутистического спектра',
           'Строка 13':'с инвалидностью вследствие  других причин','Строка 14':'из общей численности выпускников из числа лиц с ОВЗ, инвалидов и детей-инвалидов (из строки 06): имеют договор о целевом обучении',
           'Строка 15':'Узнать название ячейки',
          }
finish_df['Наименование показателей (категория выпускников)'] = finish_df['Наименование показателей (категория выпускников)'].apply(lambda x:dct[x])

In [ ]:
finish_df.to_excel('finisj.xlsx',index=False)